In [1]:
from typing import Dict, List, Optional, Tuple, Union
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage
)
from langgraph.graph import END, StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolExecutor
import json
from dataclasses import dataclass
from pydantic import BaseModel, Field

# Data Models
class Question(BaseModel):
    question: str
    answer: str
    imageUrl: Optional[str] = None

class PersonaConfig(BaseModel):
    name: str
    description: Optional[str] = None

class TeachingMethodConfig(BaseModel):
    name: str
    description: Optional[str] = None
    instructions: Optional[str] = None

class UnitConfig(BaseModel):
    chapter: str
    subChapter: str
    lesson: str
    description: str

class AgentConfig(BaseModel):
    modelName: str = "gpt-4"
    temperature: float = 1.0
    persona: PersonaConfig
    teachingMethod: TeachingMethodConfig
    selectedQuestions: List[Question]
    unit: UnitConfig

class ProgressStats(BaseModel):
    totalQuestions: int
    currentQuestion: int
    correctAnswers: int
    wrongAnswers: int
    totalAttempts: int
    averageAttemptsPerQuestion: float
    successRate: float

@dataclass
class AgentState:
    """State for the teaching agent."""
    messages: List[BaseMessage]
    current_question_index: int = 0
    correct_answers: int = 0
    wrong_answers: int = 0
    attempts: Dict[int, int] = Field(default_factory=dict)
    is_ready_to_start: bool = False
    is_complete: bool = False

class TeachingAgent:
    def __init__(self, config: AgentConfig):
        self.config = config
        self.model = ChatOpenAI(
            model_name=config.modelName,
            temperature=config.temperature
        )
        
        # Create the ask human tool
        @tool
        def ask_human(input_str: str) -> str:
            """Tool to ask the human a question."""
            return input_str

        self.tools = [ask_human]
        self.tool_executor = ToolExecutor(self.tools)
        self.model_with_tools = self.model.bind_tools(self.tools)
        
        # Initialize the state graph
        self.workflow = self._create_graph()

    def _create_graph(self) -> StateGraph:
        """Create the state graph for the agent."""
        workflow = StateGraph(AgentState)
        
        # Add nodes
        workflow.add_node("agent", self._call_model)
        workflow.add_node("action", self.tool_executor)
        workflow.add_node("process_human", self._handle_user_input)
        
        # Add edges
        workflow.add_edge("agent", "action")
        workflow.add_edge("action", "process_human")
        workflow.add_edge("process_human", "agent")
        
        # Set entry point
        workflow.set_entry_point("agent")
        
        return workflow.compile()

    def _call_model(self, state: AgentState) -> Dict:
        """Call the model with the current state."""
        system_message = SystemMessage(
            content=f"""You are teacher {self.config.persona.name}.
            Teaching method: {self.config.teachingMethod.description}
            
            Always use the ask_human tool to get student responses.
            Never respond directly - always use the tool."""
        )
        
        messages = [system_message] + state.messages
        response = self.model_with_tools.invoke(messages)
        
        return {"messages": state.messages + [response]}

    def _handle_user_input(self, state: AgentState) -> Dict:
        """Process user input and generate appropriate response."""
        user_input = state.messages[-1].content
        
        # Handle completed state
        if state.is_complete:
            response = self._generate_response("class_ended", state)
            return {"messages": state.messages + [response]}
        
        # Handle initial greeting
        if not state.is_ready_to_start:
            readiness = self._check_learning_readiness(user_input, state)
            if readiness["is_ready"]:
                state.is_ready_to_start = True
                response = self._generate_response("first_question", state)
            else:
                response = readiness["response"]
            return {"messages": state.messages + [AIMessage(content=response)]}
        
        # Handle answer checking
        current_question = self.config.selectedQuestions[state.current_question_index]
        state.attempts[state.current_question_index] = state.attempts.get(state.current_question_index, 0) + 1
        
        is_correct = self._check_answer(user_input, current_question)
        
        if not is_correct:
            state.wrong_answers += 1
            response = self._generate_response("wrong_answer_with_hint", state)
            return {"messages": state.messages + [AIMessage(content=response)]}
        
        # Handle correct answer
        state.correct_answers += 1
        state.current_question_index += 1
        
        # Check if all questions are completed
        if state.current_question_index >= len(self.config.selectedQuestions):
            state.is_complete = True
            response = self._generate_response("complete", state)
        else:
            response = self._generate_response("correct_answer", state)
            
        return {"messages": state.messages + [AIMessage(content=response)]}

    def _check_learning_readiness(self, user_input: str, state: AgentState) -> Dict:
        """Check if student is ready to start learning."""
        if state.is_ready_to_start:
            current_question = self.config.selectedQuestions[state.current_question_index]
            return {
                "is_ready": True,
                "response": f"Let's solve this problem: {current_question.question}"
            }

        # Analyze readiness using the model
        prompt = SystemMessage(
            content=f"""Previous message: "{state.messages[-1].content if state.messages else ''}"
            Student response: "{user_input}"
            Determine if the student's response is positive.
            Format: {{"isReady": true/false}}"""
        )
        
        result = self.model.invoke([prompt])
        analysis = json.loads(result.content)
        
        # Generate response based on readiness
        if analysis["isReady"]:
            state.is_ready_to_start = True
            response = self._generate_response("first_question", state)
        else:
            response = f"Hello! I'm {self.config.persona.name}. We'll be studying {self.config.unit.chapter} {self.config.unit.lesson}. Shall we begin?"
            
        return {
            "is_ready": analysis["isReady"],
            "response": response
        }

    def _check_answer(self, user_input: str, question: Question) -> bool:
        """Check if the student's answer is correct."""
        prompt = SystemMessage(
            content=f"""Question: "{question.question}"
            Student answer: "{user_input}"
            Expected answer: "{question.answer}"
            
            Analyze if the answer is correct based on:
            1. Core concept understanding
            2. Context appropriateness
            3. Semantic similarity to expected answer
            
            Format: {{
                "isCorrect": true/false,
                "similarity": "high/medium/low",
                "reason": "explanation"
            }}"""
        )
        
        result = self.model.invoke([prompt])
        analysis = json.loads(result.content)
        
        return analysis["similarity"] in ["high", "medium"]

    def _generate_response(self, situation: str, state: AgentState) -> str:
        """Generate appropriate response based on the situation."""
        # Implementation similar to TypeScript version
        # Returns formatted response string
        pass  # Full implementation would go here

    def start(self) -> str:
        """Start the teaching session."""
        greeting = self._generate_response("greeting", AgentState(messages=[]))
        return greeting

    def process_message(self, message: str) -> str:
        """Process a message from the student."""
        state = AgentState(messages=[HumanMessage(content=message)])
        result = self.workflow(state)
        return result.messages[-1].content

# Example usage
if __name__ == "__main__":
    # Sample configuration
    config = AgentConfig(
        persona=PersonaConfig(
            name="Ms. Smith",
            description="Friendly elementary school teacher"
        ),
        teachingMethod=TeachingMethodConfig(
            name="Socratic Method",
            description="Guide through questioning"
        ),
        selectedQuestions=[
            Question(
                question="What is 2 + 2? (Think about counting)",
                answer="4"
            ),
            Question(
                question="What is 5 - 3? (Use objects to help)",
                answer="2"
            )
        ],
        unit=UnitConfig(
            chapter="Basic Math",
            subChapter="Addition and Subtraction",
            lesson="Lesson 1",
            description="Introduction to basic operations"
        )
    )
    
    # Create and start the agent
    agent = TeachingAgent(config)
    
    # Start conversation
    response = agent.start()
    print("Agent:", response)
    
    # Example interaction
    while True:
        user_input = input("Student: ")
        if user_input.lower() in ["exit", "quit"]:
            break
            
        response = agent.process_message(user_input)
        print("Agent:", response)

C:\Users\DNSOFT\AppData\Local\Temp\ipykernel_17420\131266387.py:80: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  self.tool_executor = ToolExecutor(self.tools)


Agent: None


TypeError: 'CompiledStateGraph' object is not callable

In [6]:
from typing import Dict, List, Optional, Tuple, Union, Literal
from typing_extensions import TypeAlias

# Define type for graph steps
GraphStep: TypeAlias = Union[Literal["action", "process_human", "end"], type(END)]
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage
)
from langgraph.graph import END, StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolExecutor
import json
from dataclasses import dataclass
from pydantic import BaseModel, Field

# Data Models
class Question(BaseModel):
    question: str
    answer: str
    imageUrl: Optional[str] = None

class PersonaConfig(BaseModel):
    name: str
    description: Optional[str] = None

class TeachingMethodConfig(BaseModel):
    name: str
    description: Optional[str] = None
    instructions: Optional[str] = None

class UnitConfig(BaseModel):
    chapter: str
    subChapter: str
    lesson: str
    description: str

class AgentConfig(BaseModel):
    modelName: str = "gpt-4"
    temperature: float = 1.0
    persona: PersonaConfig
    teachingMethod: TeachingMethodConfig
    selectedQuestions: List[Question]
    unit: UnitConfig

class ProgressStats(BaseModel):
    totalQuestions: int
    currentQuestion: int
    correctAnswers: int
    wrongAnswers: int
    totalAttempts: int
    averageAttemptsPerQuestion: float
    successRate: float

@dataclass
class AgentState(BaseModel):
    """State for the teaching agent."""
    messages: List[BaseMessage] = []
    current_question_index: int = 0
    correct_answers: int = 0
    wrong_answers: int = 0
    attempts: Dict[int, int] = {}
    is_ready_to_start: bool = False
    is_complete: bool = False
    tool: Optional[str] = None
    tool_input: Optional[str] = None
    tool_response: Optional[str] = None

    model_config = {
        "arbitrary_types_allowed": True,
        "validate_assignment": True
    }

    def __init__(self, **data):
        super().__init__(**data)
        if not self.messages:
            self.messages = []
        if not self.attempts:
            self.attempts = {}

class TeachingAgent:
    def __init__(self, config: AgentConfig):
        self.config = config
        self.model = ChatOpenAI(
            model_name=config.modelName,
            temperature=config.temperature
        )
        
        # Create the ask human tool
        @tool
        def ask_human(input_str: str) -> str:
            """Tool to ask the human a question."""
            return input_str

        self.tools = [ask_human]
        self.tool_executor = ToolExecutor(self.tools)
        self.model_with_tools = self.model.bind_tools(self.tools)
        
        # Initialize the state graph
        self.workflow = self._create_graph()

    def _create_graph(self) -> StateGraph:
        """Create the state graph for the agent."""
        workflow = StateGraph(AgentState)
        
        # Add nodes
        workflow.add_node("agent", self._call_model)
        workflow.add_node("action", self.tool_executor)
        workflow.add_node("process_human", self._handle_user_input)
        
        # Add conditional edges
        workflow.add_conditional_edges(
            "agent",
            self._route_next_step,
            {
                "action": "action",
                "process_human": "process_human",
                "end": END
            }
        )
        workflow.add_edge("action", "agent")
        workflow.add_edge("process_human", "agent")
        
        # Set entry point
        workflow.set_entry_point("agent")
        
        return workflow.compile()

    def _call_model(self, state: AgentState) -> Dict:
        """Call the model with the current state."""
        system_message = SystemMessage(
            content=f"""당신은 {self.config.persona.name} 선생님입니다.
            교수법: {self.config.teachingMethod.description}
            
            반드시 ask_human 도구를 사용하여 학생의 응답을 받으세요.
            직접 대화하지 말고 항상 ask_human 도구를 통해 대화하세요."""
        )
        
        messages = [system_message] + state.messages
        response = self.model_with_tools.invoke(messages)
        
        return {"messages": state.messages + [response], "state": state}

    def _handle_user_input(self, state: AgentState) -> Dict:
        """Process user input and generate appropriate response."""
        user_input = state.messages[-1].content
        
        # Handle completed state
        if state.is_complete:
            response = self._generate_response("class_ended", state)
            return {"messages": state.messages + [response]}
        
        # Handle initial greeting
        if not state.is_ready_to_start:
            readiness = self._check_learning_readiness(user_input, state)
            if readiness["is_ready"]:
                state.is_ready_to_start = True
                response = self._generate_response("first_question", state)
            else:
                response = readiness["response"]
            return {"messages": state.messages + [AIMessage(content=response)]}
        
        # Handle answer checking
        current_question = self.config.selectedQuestions[state.current_question_index]
        state.attempts[state.current_question_index] = state.attempts.get(state.current_question_index, 0) + 1
        
        is_correct = self._check_answer(user_input, current_question)
        
        if not is_correct:
            state.wrong_answers += 1
            response = self._generate_response("wrong_answer_with_hint", state)
            return {"messages": state.messages + [AIMessage(content=response)]}
        
        # Handle correct answer
        state.correct_answers += 1
        state.current_question_index += 1
        
        # Check if all questions are completed
        if state.current_question_index >= len(self.config.selectedQuestions):
            state.is_complete = True
            response = self._generate_response("complete", state)
        else:
            response = self._generate_response("correct_answer", state)
            
        return {"messages": state.messages + [AIMessage(content=response)]}

    def _check_learning_readiness(self, user_input: str, state: AgentState) -> Dict:
        """Check if student is ready to start learning."""
        if state.is_ready_to_start:
            current_question = self.config.selectedQuestions[state.current_question_index]
            return {
                "is_ready": True,
                "response": f"Let's solve this problem: {current_question.question}"
            }

        # Analyze readiness using the model
        prompt = SystemMessage(
            content=f"""Previous message: "{state.messages[-1].content if state.messages else ''}"
            Student response: "{user_input}"
            Determine if the student's response is positive.
            Format: {{"isReady": true/false}}"""
        )
        
        result = self.model.invoke([prompt])
        analysis = json.loads(result.content)
        
        # Generate response based on readiness
        if analysis["isReady"]:
            state.is_ready_to_start = True
            response = self._generate_response("first_question", state)
        else:
            response = f"Hello! I'm {self.config.persona.name}. We'll be studying {self.config.unit.chapter} {self.config.unit.lesson}. Shall we begin?"
            
        return {
            "is_ready": analysis["isReady"],
            "response": response
        }

    def _check_answer(self, user_input: str, question: Question) -> bool:
        """Check if the student's answer is correct."""
        prompt = SystemMessage(
            content=f"""Question: "{question.question}"
            Student answer: "{user_input}"
            Expected answer: "{question.answer}"
            
            Analyze if the answer is correct based on:
            1. Core concept understanding
            2. Context appropriateness
            3. Semantic similarity to expected answer
            
            Format: {{
                "isCorrect": true/false,
                "similarity": "high/medium/low",
                "reason": "explanation"
            }}"""
        )
        
        result = self.model.invoke([prompt])
        analysis = json.loads(result.content)
        
        return analysis["similarity"] in ["high", "medium"]

    def _generate_response(self, situation: str, state: AgentState) -> str:
        """Generate appropriate response based on the situation."""
        prompt = f"""당신은 {self.config.persona.name} 선생님입니다.
        교수법: {self.config.teachingMethod.description}
        현재 수업: {self.config.unit.chapter} - {self.config.unit.lesson}

        다음 상황에 대한 응답을 생성하세요: {situation}"""

        if situation == "greeting":
            prompt += "\n친근한 인사와 자기소개를 해주세요."
        elif situation == "first_question":
            current_question = self.config.selectedQuestions[0]
            prompt += f"\n다음 문제를 제시해주세요: {current_question.question}"
        elif situation == "correct_answer":
            current_question = self.config.selectedQuestions[state.current_question_index]
            prompt += f"\n학생이 정답을 맞췄습니다. 다음 문제를 제시해주세요: {current_question.question}"
        elif situation == "wrong_answer_with_hint":
            current_question = self.config.selectedQuestions[state.current_question_index]
            prompt += f"\n학생이 틀렸습니다. 격려와 함께 다음 문제에 대한 힌트를 제공해주세요: {current_question.question}"
        elif situation == "complete":
            stats = self._get_progress_stats(state)
            prompt += f"\n모든 문제가 완료되었습니다. 성과를 요약해주세요: {stats}"
        elif situation == "class_ended":
            prompt += "\n친근한 마무리 인사를 해주세요."

        response = self.model.invoke([SystemMessage(content=prompt)])
        return response.content

    def start(self) -> str:
        """Start the teaching session."""
        initial_state = AgentState()  # All fields will use defaults
        greeting = self._generate_response("greeting", initial_state)
        return greeting

    def process_message(self, message: str) -> str:
        """Process a message from the student."""
        initial_state = AgentState(messages=[HumanMessage(content=message)])
        inputs = {"messages": initial_state.messages}
        for output in self.workflow.stream(inputs):
            if "messages" in output:
                initial_state.messages = output["messages"]
        return initial_state.messages[-1].content if initial_state.messages else "No response generated."

    def _route_next_step(self, state: AgentState) -> GraphStep:
        """Determine the next step in the workflow."""
        last_message = state.messages[-1]
        if isinstance(last_message, AIMessage) and last_message.additional_kwargs.get("tool_calls"):
            return "action"
        elif isinstance(last_message, HumanMessage):
            return "process_human"
        return "end"

# Example usage
if __name__ == "__main__":
    # Sample configuration
    config = AgentConfig(
        persona=PersonaConfig(
            name="Ms. Smith",
            description="Friendly elementary school teacher"
        ),
        teachingMethod=TeachingMethodConfig(
            name="Socratic Method",
            description="Guide through questioning"
        ),
        selectedQuestions=[
            Question(
                question="What is 2 + 2? (Think about counting)",
                answer="4"
            ),
            Question(
                question="What is 5 - 3? (Use objects to help)",
                answer="2"
            )
        ],
        unit=UnitConfig(
            chapter="Basic Math",
            subChapter="Addition and Subtraction",
            lesson="Lesson 1",
            description="Introduction to basic operations"
        )
    )
    
    # Create and start the agent
    agent = TeachingAgent(config)
    
    # Start conversation
    response = agent.start()
    print("Agent:", response)
    
    # Example interaction
    while True:
        user_input = input("Student: ")
        if user_input.lower() in ["exit", "quit"]:
            break
            
        response = agent.process_message(user_input)
        print("Agent:", response)

C:\Users\DNSOFT\AppData\Local\Temp\ipykernel_17420\3466432966.py:99: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  self.tool_executor = ToolExecutor(self.tools)


Agent: 안녕하세요, 여러분! 여러분을 모두 만나게 되어 기쁩니다. 저는 당신들의 수학 선생님, 스미스라고 합니다. 우리가 함께할 이 시간 동안, 저는 여러분에게 질문을 통해 학습을 안내할 것입니다. 많은 질문과 참여를 기대합니다. 다들 준비가 되셨나요? 시작해볼까요, 여러분?
Agent: yes 
Agent: ??


InvalidUpdateError: Expected dict, got None is not a valid tool, try one of [ask_human].
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

In [16]:
from typing import Dict, List, Optional, Union
from langchain_core.messages import BaseMessage, AIMessage, SystemMessage, HumanMessage, FunctionMessage
from langgraph.graph import StateGraph, END, START
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool, tool
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel, Field
import json

class Question(BaseModel):
    question: str
    answer: str
    imageUrl: Optional[str] = None

class AgentState(BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    current_question_index: int = 0
    correct_answers: int = 0
    wrong_answers: int = 0
    attempts: Dict[int, int] = Field(default_factory=dict)
    is_ready_to_start: bool = False
    is_complete: bool = False
    current_tool_calls: List[dict] = Field(default_factory=list)

    class Config:
        arbitrary_types_allowed = True

class TeachingAgent:
    @tool
    def ask_human(input_str: str) -> str:
        """학생에게 질문하거나 응답을 요청할 때 사용합니다."""
        return input_str

    def __init__(self):
        self.persona = {
            "name": "김선생님",
            "description": "친근한 초등학교 선생님"
        }
        self.teaching_method = {
            "name": "소크라테스식 교수법",
            "description": "질문을 통한 학습 유도"
        }
        self.unit = {
            "chapter": "기초 수학",
            "subChapter": "덧셈과 뺄셈",
            "lesson": "1차시",
            "description": "기본 연산 소개"
        }
        
        self.selected_questions = [
            Question(
                question="2 + 2는 얼마일까요? (손가락으로 세어보세요)",
                answer="4"
            ),
            Question(
                question="5 - 3은 얼마일까요? (물건을 사용해서 계산해보세요)",
                answer="2"
            )
        ]
        
        self.model = ChatOpenAI(
            model_name="gpt-4",
            temperature=1.0
        )

        self.model_with_tools = self.model.bind_tools([TeachingAgent.ask_human])
        self.state = AgentState()
        self.workflow = self._create_workflow()

    def _create_workflow(self) -> StateGraph:
        workflow = StateGraph(AgentState)
        
        workflow.add_node("agent", self._call_model)
        workflow.add_node("human", self._handle_human_input)
        
        workflow.add_conditional_edges(
            "agent",
            self._should_continue,
            {
                "human": "human",
                "end": END
            }
        )
        
        workflow.add_edge("human", "agent")
        workflow.add_edge(START, "agent")
        
        return workflow.compile()

    def _should_continue(self, state: AgentState) -> str:
        """Determine if we should continue the conversation."""
        last_msg = state.messages[-1] if state.messages else None
        
        if isinstance(last_msg, HumanMessage):
            return "human"
        elif isinstance(last_msg, AIMessage):
            if last_msg.additional_kwargs.get("tool_calls"):
                return "human"
            
        return "end"

    def _call_model(self, state: AgentState) -> Dict:
        """Handle model calls and responses."""
        system_msg = SystemMessage(content=f"""당신은 {self.persona['name']} 선생님입니다.
        교수법: {self.teaching_method['description']}
        
        반드시 ask_human 도구를 사용하여 학생의 응답을 받으세요.
        직접 대화하지 말고 항상 ask_human 도구를 통해 대화하세요.""")
        
        messages = [system_msg] + state.messages[-5:]  # Keep last 5 messages for context
        
        try:
            response = self.model_with_tools.invoke(messages)
            
            # If there are tool calls, save them for later
            if response.additional_kwargs.get("tool_calls"):
                state.current_tool_calls = response.additional_kwargs["tool_calls"]
            
            return {"messages": state.messages + [response]}
        except Exception as e:
            print(f"Model error: {e}")
            return {"messages": state.messages + [AIMessage(content="죄송합니다. 다시 시도해주세요.")]}

    def _handle_human_input(self, state: AgentState) -> Dict:
        """Process human input and tool responses."""
        if not state.messages:
            return {"messages": state.messages}

        last_message = state.messages[-1]
        
        # If we have pending tool calls, handle them
        if state.current_tool_calls:
            for tool_call in state.current_tool_calls:
                # Add tool response
                state.messages.append(
                    FunctionMessage(
                        content=last_message.content,
                        name="ask_human",
                        additional_kwargs={"tool_call_id": tool_call["id"]}
                    )
                )
            state.current_tool_calls = []  # Clear pending tool calls
            
        # Check if this is an answer to a question
        if state.current_question_index < len(self.selected_questions):
            current_question = self.selected_questions[state.current_question_index]
            
            # Evaluate answer
            prompt = SystemMessage(content=f"""문제: "{current_question.question}"
                학생 답변: "{last_message.content}"
                기대하는 답변: "{current_question.answer}"
                
                학생의 답변이 정답과 일치하나요?
                형식: {{"isCorrect": true/false}}""")
            
            result = self.model.invoke([prompt])
            try:
                analysis = json.loads(result.content)
                if analysis.get("isCorrect"):
                    state.correct_answers += 1
                    state.current_question_index += 1
                    if state.current_question_index < len(self.selected_questions):
                        response = f"잘했어요! 다음 문제입니다: {self.selected_questions[state.current_question_index].question}"
                    else:
                        state.is_complete = True
                        response = "모든 문제를 다 풀었습니다! 정말 잘했어요!"
                else:
                    state.wrong_answers += 1
                    response = f"다시 한번 생각해볼까요? {current_question.question}"
                    
                state.messages.append(AIMessage(content=response))
                return {"messages": state.messages}
            except:
                pass
        
        return {"messages": state.messages}

    def process_message(self, message: str) -> str:
        """Process a message and return the response."""
        if self.state.is_complete:
            return "수업이 이미 종료되었습니다."
            
        self.state.messages.append(HumanMessage(content=message))
        
        try:
            for output in self.workflow.stream({"messages": self.state.messages}):
                if "messages" in output:
                    self.state.messages = output["messages"]
            
            # Get the last AI message
            for msg in reversed(self.state.messages):
                if isinstance(msg, AIMessage):
                    return msg.content
                    
            return "응답을 생성할 수 없습니다."
        except Exception as e:
            print(f"Error in process_message: {e}")
            return "오류가 발생했습니다. 다시 시도해주세요."

    def start(self) -> str:
        """Start the teaching session."""
        greeting = f"""안녕하세요! 저는 {self.persona['name']}입니다.
        오늘은 {self.unit['chapter']} {self.unit['lesson']}을 함께 공부할 예정입니다.
        첫 번째 문제입니다: {self.selected_questions[0].question}"""
        
        self.state.messages = [AIMessage(content=greeting)]
        return greeting

# Example usage
if __name__ == "__main__":
    agent = TeachingAgent()
    print("선생님:", agent.start())
    
    while True:
        user_input = input("학생: ")
        if user_input.lower() in ["종료", "quit", "exit"]:
            break
            
        response = agent.process_message(user_input)
        print("선생님:", response)

선생님: 안녕하세요! 저는 김선생님입니다.
        오늘은 기초 수학 1차시을 함께 공부할 예정입니다.
        첫 번째 문제입니다: 2 + 2는 얼마일까요? (손가락으로 세어보세요)
Model error: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_LqXDlV4kdVopxBkqTvU3VdTe", 'type': 'invalid_request_error', 'param': 'messages.[4].role', 'code': None}}
선생님: 안녕하세요! 저는 김선생님입니다.
        오늘은 기초 수학 1차시을 함께 공부할 예정입니다.
        첫 번째 문제입니다: 2 + 2는 얼마일까요? (손가락으로 세어보세요)
Model error: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_ULql0dFq77taiMpcEA7a2sdo", 'type': 'invalid_request_error', 'param': 'messages.[5].role', 'code': None}}
선생님: 안녕하세요! 저는 김선생님입니다.
        오늘은 기초 수학 1차시을 함께 공부할 예정입니다.
        첫 번째 문제입니다: 2 + 2는 얼마일까요? (손가락으로 세어보세요)
Model err

KeyboardInterrupt: 

In [1]:
from typing import Dict, List, Optional, Union
from langchain_core.messages import BaseMessage, AIMessage, SystemMessage, HumanMessage, FunctionMessage
from langgraph.graph import StateGraph, END, START
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool, tool
from pydantic import BaseModel, Field
import json

class Question(BaseModel):
    question: str
    answer: str
    imageUrl: Optional[str] = None

class AgentState(BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    current_question_index: int = 0
    correct_answers: int = 0
    wrong_answers: int = 0
    attempts: Dict[int, int] = Field(default_factory=dict)
    is_complete: bool = False

    class Config:
        arbitrary_types_allowed = True

class TeachingAgent:
    def __init__(self):
        self.persona = {
            "name": "김선생님",
            "description": "친근한 초등학교 선생님"
        }
        self.teaching_method = {
            "name": "소크라테스식 교수법",
            "description": "질문을 통한 학습 유도"
        }
        self.unit = {
            "chapter": "기초 수학",
            "subChapter": "덧셈과 뺄셈",
            "lesson": "1차시",
            "description": "기본 연산 소개"
        }
        
        self.selected_questions = [
            Question(
                question="2 + 2는 얼마일까요? (손가락으로 세어보세요)",
                answer="4"
            ),
            Question(
                question="5 - 3은 얼마일까요? (물건을 사용해서 계산해보세요)",
                answer="2"
            )
        ]
        
        self.model = ChatOpenAI(
            model_name="gpt-4",
            temperature=1.0
        )
        self.state = AgentState()

    def evaluate_answer(self, user_input: str, current_question: Question) -> bool:
        """Evaluate if the answer is correct."""
        prompt = SystemMessage(content=f"""문제: "{current_question.question}"
            학생 답변: "{user_input}"
            기대하는 답변: "{current_question.answer}"
            
            학생의 답변이 정답과 일치하나요?
            형식: {{"isCorrect": true/false}}""")
        
        result = self.model.invoke([prompt])
        try:
            analysis = json.loads(result.content)
            return analysis.get("isCorrect", False)
        except:
            return False

    def process_message(self, message: str) -> str:
        """Process a user message and return response."""
        if self.state.is_complete:
            return "수업이 이미 종료되었습니다."

        # Handle first message
        if not self.state.messages:
            return self.start()
            
        # Get current question
        current_question = self.selected_questions[self.state.current_question_index]
        
        # Check answer
        is_correct = self.evaluate_answer(message, current_question)
        
        if is_correct:
            self.state.correct_answers += 1
            self.state.current_question_index += 1
            
            if self.state.current_question_index >= len(self.selected_questions):
                self.state.is_complete = True
                self.state.messages.append(HumanMessage(content=message))
                stats = self.get_stats()
                return f"""정답입니다! 모든 문제를 다 풀었습니다!
                
학습 결과:
- 총 {stats['total_questions']}개의 문제를 풀었습니다.
- 정답: {stats['correct_answers']}개
- 오답: {stats['wrong_answers']}개
- 정답률: {stats['success_rate']}%

정말 잘했어요! 다음 수업에서 만나요!"""
            else:
                next_question = self.selected_questions[self.state.current_question_index]
                self.state.messages.append(HumanMessage(content=message))
                return f"정답입니다! 다음 문제입니다: {next_question.question}"
        else:
            self.state.wrong_answers += 1
            self.state.messages.append(HumanMessage(content=message))
            return f"다시 한번 생각해볼까요? {current_question.question}"

    def start(self) -> str:
        """Start the teaching session."""
        greeting = f"""안녕하세요! 저는 {self.persona['name']}입니다.
오늘은 {self.unit['chapter']} {self.unit['lesson']}을 함께 공부할 예정입니다.
첫 번째 문제입니다: {self.selected_questions[0].question}"""
        
        self.state.messages = [AIMessage(content=greeting)]
        return greeting

    def get_stats(self) -> Dict:
        """Get current statistics."""
        total_questions = len(self.selected_questions)
        return {
            'total_questions': total_questions,
            'correct_answers': self.state.correct_answers,
            'wrong_answers': self.state.wrong_answers,
            'success_rate': round((self.state.correct_answers / total_questions) * 100, 1)
        }

# Example usage
if __name__ == "__main__":
    agent = TeachingAgent()
    print("선생님:", agent.start())
    
    while True:
        user_input = input("학생: ")
        if user_input.lower() in ["종료", "quit", "exit"]:
            break
            
        response = agent.process_message(user_input)
        print("선생님:", response)

선생님: 안녕하세요! 저는 김선생님입니다.
오늘은 기초 수학 1차시을 함께 공부할 예정입니다.
첫 번째 문제입니다: 2 + 2는 얼마일까요? (손가락으로 세어보세요)
선생님: 다시 한번 생각해볼까요? 2 + 2는 얼마일까요? (손가락으로 세어보세요)
선생님: 다시 한번 생각해볼까요? 2 + 2는 얼마일까요? (손가락으로 세어보세요)
선생님: 정답입니다! 다음 문제입니다: 5 - 3은 얼마일까요? (물건을 사용해서 계산해보세요)
선생님: 다시 한번 생각해볼까요? 5 - 3은 얼마일까요? (물건을 사용해서 계산해보세요)
선생님: 정답입니다! 모든 문제를 다 풀었습니다!
                
학습 결과:
- 총 2개의 문제를 풀었습니다.
- 정답: 2개
- 오답: 3개
- 정답률: 100.0%

정말 잘했어요! 다음 수업에서 만나요!
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
선생님: 수업이 이미 종료되었습니다.
